In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from lazypredict.Supervised import LazyRegressor


In [2]:
df = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI Collection and Loss Reverse Forecast\modelling\kolektor\regresi_for_time\data\kolektor.csv")
df.sample(5)

,Unnamed: 0,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,negotiation_skills,collector_experience
417,417,9,surat panggilan,30,229500000,429500000,2879807,53.00,Buruk,0
688,688,29,datang ke tempat,33,506000000,927000000,62652541,55.00,Cukup baik,2
96,96,6,surat panggilan,23,55500000,58000000,1438630,96.00,Baik,4
64,64,15,panggilan,36,391000000,625000000,28670220,63.00,Baik,4
542,542,3,sms or WA,15,25900000,29600000,312001,88.00,Baik,4


In [3]:
for i in df.columns:
    if df[i].dtype == "object":
        df[i] = df[i].astype("category")

df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [4]:
df.head()

,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,negotiation_skills,collector_experience
0,0,sms or WA,7,14400000,35700000,277706,40.00,Baik,4
1,11,panggilan,35,79000000,160500000,22382611,49.00,Buruk,0
2,4,sms or WA,6,7100000,26700000,205389,27.00,Buruk,0
3,29,datang ke tempat,63,782000000,1395000000,482942548,56.00,Cukup baik,2
4,22,datang ke tempat,35,346000000,485500000,151035603,71.00,Cukup baik,2


In [5]:
bill_methods = {"sms or WA": 0, "surat panggilan": 1, "panggilan": 2, "datang ke tempat": 3}
nego = {"Buruk": 0, "Cukup baik": 1, "Baik": 2}
columns = {"avg_bill_methods": bill_methods, "negotiation_skills": nego}

for i in df.columns:
    if df[i].dtype == "category" or df[i].dtype == "object":
        df[i] = [columns[i][j] for j in df[i]]

             
scaler = RobustScaler().fit(df[[i for i in df.columns if i not in columns]])
df2 = pd.DataFrame(scaler.transform(df[[i for i in df.columns if i not in columns]]), columns=df[[i for i in df.columns if i not in columns]].columns)
for i in columns:
    df2[i] = df[i]
df2.head(5)

,time_to_collect,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,collector_experience,avg_bill_methods,negotiation_skills
0,-1.00,-1.35,-0.46,-0.52,-0.25,-0.36,0.50,0,2
1,-0.27,-0.13,-0.28,-0.34,-0.16,-0.16,-0.50,2,0
2,-0.73,-1.39,-0.48,-0.53,-0.25,-0.66,-0.50,0,0
3,0.93,1.09,1.64,1.39,1.72,0.00,0.00,3,1
4,0.47,-0.13,0.45,0.11,0.36,0.34,0.00,3,1


In [6]:
X = df2.drop("total_cost", axis=1)
y = df2["total_cost"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:07<00:00,  5.50it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001179 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 718
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 8
[LightGBM] [Info] Start training from score 0.296629
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
AdaBoostRegressor,0.67,0.68,0.40,0.06
MLPRegressor,0.65,0.67,0.41,0.51
RandomForestRegressor,0.64,0.66,0.42,1.56
BaggingRegressor,0.64,0.65,0.42,0.13
NuSVR,0.62,0.64,0.43,0.09
SVR,0.62,0.63,0.43,0.11
GradientBoostingRegressor,0.61,0.62,0.44,0.48
ExtraTreesRegressor,0.60,0.62,0.44,0.87
OrthogonalMatchingPursuitCV,0.56,0.58,0.46,0.03
